<a href="https://colab.research.google.com/github/jlopetegui98/Creation-of-a-synthetic-dataset-for-French-NER-in-clinical-trial-texts/blob/main/NER-chia-dataset/model_ner_chia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Multilingual NER model trained over [Chia dataset](https://figshare.com/articles/dataset/Chia_Annotated_Datasets/11855817)**

We are going to train a BERT based multilingual language model over the Chia dataset in english and then we will use this model to create the synthetic version of the dataset in French. Our idea is supported by the experiments already done with [multiNERD](https://huggingface.co/datasets/Babelscape/multinerd) dataset for multilingual NER in English and French.

**Entities selection**

Among all the entities in the dataset, we are going to focus for this project on the most represented ones. Then, we are just going to consider those entities with more than 1000 samples in total.

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2ba57b3619c540aef935bfd39ccf0633e44938ac090687fb5464ea16e3478d71
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
# imports
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import os
# from preprocessing_dataset import *
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
import json
from datasets.features import ClassLabel

In [5]:
# dict for the entities (entity to int value)
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Grug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16
}
entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [6]:
# data paths
root_path = './' # comment if working on colab
root_path = './drive/MyDrive/HandsOn-NLP'
data_path = f'{root_path}/data'
chia_bio_path = f"{data_path}/chia_bio"
chia_prep_path = f"{data_path}/chia_prep"
models_path = f"{root_path}/models"

In [7]:
# preprocessing dataset to get the data in the right format for dataset entity creation
# preprocessing_dataset(chia_bio_path, output_path=chia_prep_path, labels2int = sel_ent, int2labels = sel_ent_inv)

In [8]:
# read the data after preprocessing
files = os.listdir(chia_prep_path)
files[:5]

['NCT02893293_exc.bio.json',
 'NCT03475589_exc.bio.json',
 'NCT01997580_inc.bio.json',
 'NCT01993836_exc.bio.json',
 'NCT02589691_inc.bio.json']

In [9]:
sentences = []

for file in files:
    with open(f"{chia_prep_path}/{file}", "r") as f:
        stc = json.load(f)
        sentences.extend(stc["sentences"])

In [10]:
# create the dataset
chia_eng_dataset = Dataset.from_pandas(pd.DataFrame(sentences))

In [11]:
chia_eng_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 12423
})

In [12]:
chia_eng_train_test = chia_eng_dataset.train_test_split(test_size=0.2)
chia_eng_test_val = chia_eng_train_test["test"].train_test_split(test_size=0.5)
chia_eng_dataset = DatasetDict({
    "train": chia_eng_train_test["train"],
    "test": chia_eng_test_val["test"],
    "validation": chia_eng_test_val["train"]
})

In [13]:
chia_eng_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9938
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1243
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1242
    })
})

**Model Implementation**

In [14]:
model_name = 'xlm-roberta-base'

In [15]:
# get xlm-roberta tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# check the tokenizer
tokens_ = tokenizer("The AI master at Université Paris-Saclay is very good").tokens()
print(tokens_)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

['<s>', '▁The', '▁AI', '▁master', '▁at', '▁', 'Université', '▁Paris', '-', 'S', 'ac', 'lay', '▁is', '▁very', '▁good', '</s>']


In [16]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

In [17]:
type(chia_eng_dataset)

datasets.dataset_dict.DatasetDict

In [18]:
# apply the function to the dataset
chia_eng_dataset = chia_eng_dataset.map(tokenize_and_align_labels, batched=True)
chia_eng_dataset

Map:   0%|          | 0/9938 [00:00<?, ? examples/s]

Map:   0%|          | 0/1243 [00:00<?, ? examples/s]

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9938
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1243
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1242
    })
})

In [19]:
# import the model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(entities_list), label2id=sel_ent, id2label=sel_ent_inv)
print(model)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
model.to(device)

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [22]:
# define the training arguments
args = TrainingArguments(
    "chia-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps=50,
    eval_steps=50,
    save_steps=1000,
)

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [24]:
metric = load_metric("seqeval")

<ipython-input-24-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
def compute_metrics(p):
    """
    Compute the metrics for the model
    inputs:
        p: tuple, the predictions and the labels
    outputs:
        dict: the metrics
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
# define the trainer
trainer = Trainer(
    model,
    args,
    train_dataset=chia_eng_dataset["train"],
    eval_dataset=chia_eng_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
outputs_train = trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,1.900400,1.311283,0.462752,0.372847,0.412963,0.617104
100,1.194300,0.993749,0.560623,0.546213,0.553324,0.714445
150,0.973900,0.820683,0.621495,0.636451,0.628884,0.760178
200,0.811000,0.738246,0.658550,0.665573,0.662043,0.780740
250,0.776100,0.664086,0.688972,0.703856,0.696334,0.802677
300,0.736000,0.654238,0.690409,0.707684,0.698940,0.802863
350,0.675200,0.622444,0.693347,0.722450,0.707600,0.811675
400,0.683700,0.600380,0.706940,0.731200,0.718866,0.816248
450,0.616100,0.577281,0.720021,0.735576,0.727715,0.820524
500,0.645000,0.574138,0.727431,0.740361,0.733839,0.821751


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print(outputs_train)

TrainOutput(global_step=1866, training_loss=0.6053849045997775, metrics={'train_runtime': 780.1746, 'train_samples_per_second': 38.215, 'train_steps_per_second': 2.392, 'total_flos': 1193038624065696.0, 'train_loss': 0.6053849045997775, 'epoch': 3.0})


In [29]:
outputs_eval = trainer.evaluate(chia_eng_dataset["test"])

In [30]:
print(outputs_eval)

{'eval_loss': 0.45987215638160706, 'eval_precision': 0.7576590190960297, 'eval_recall': 0.8037015447391431, 'eval_f1': 0.7800014143271339, 'eval_accuracy': 0.8471634710417079, 'eval_runtime': 5.1422, 'eval_samples_per_second': 241.725, 'eval_steps_per_second': 30.337, 'epoch': 3.0}


In [31]:
torch.save(model, f"{models_path}/chia-multilingual-ner.pt")

In [35]:
model.to(device)

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [39]:
preds = trainer.predict(chia_eng_dataset["test"], metric_key_prefix="")


array([[[ 1.1799247e+00, -3.8492057e-01,  1.6270538e-01, ...,
          5.4497398e-02, -6.4639908e-01, -3.0931500e-01],
        [ 2.0732725e+00,  1.9636600e+00, -4.9360409e-01, ...,
         -1.2057234e+00, -5.4105002e-01, -8.3505541e-01],
        [-8.8526469e-01,  2.1118748e+00,  1.9313463e+00, ...,
          5.2717912e-01, -1.7728847e+00, -2.1955469e-01],
        ...,
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02]],

       [[ 1.2905122e+00, -3.1185555e-01,  1.6733825e-02, ...,
          1.6036035e-01, -4.5042732e-01, -9.6372142e-02],
        [ 8.7133551e+00, -3.0445674e-01, -4.6516836e-01, ...,
          5.9204400e-01, -5.6929495e-02, -1.2118070e+00],
        [ 8.2033014e+00, 

In [47]:
preds.label_ids[0]

array([-100,    1,    2,    2,    2,    2,    2,    2,    0,   11,   12,
         12,    0,   11,   12,   12,   12,   12,   12,   12,   12,   12,
         12,   12,    3,    9,   10,   10,   10,   10,   10,   10,   10,
         10,   10,   10,   10,   10,   10,   10,   10,   10,    0,    1,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -1

In [51]:
len(chia_eng_dataset['test'][0]['input_ids'])

45

In [57]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])

# Take the first column and stack it with the second column
result = np.column_stack(a)

result
# # Reshape the result to make it a 2D array with 3 rows
# result = result.reshape(3, 2)

# print(result)


array([[1, 4],
       [2, 5],
       [3, 6]])

In [54]:
a.reshape(3,-1)

array([[1, 2],
       [3, 4],
       [5, 6]])